In [7]:
import torch
import torch.nn as nn 
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets
import torch.nn.functional as F
import matplotlib.pyplot as plt
from DenseNet2 import DenseNet

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=True)

100%|█████████████████████████████████████████████████████████████▉| 170205184/170498071 [00:13<00:00, 10993905.09it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
## A sample training loop for the densenet implementation. To achieve high accuracy, many more training tricks must be used. 
rate = .001
torch.cuda.set_device(0)
device = torch.cuda.device(0)
loss_func = nn.CrossEntropyLoss()
net = DenseNet(8, [6,12,18,12],10, 200).cuda()
optimizer = optim.Adam(net.parameters(),lr = rate)
running_loss = 0
epochs = 20

net.train()
for e in range(epochs): 
    print(running_loss)
    running_loss = 0
    accuracy = 0
    i = 0
    for images, labels in trainloader: 
        
        images = images.cuda()
        labels = labels.cuda()
        i += 1
        
        optimizer.zero_grad()
        out = net(images)
        
        loss = loss_func(out, labels)
       
           
        loss.backward()
        optimizer.step()
        running_loss += loss
    
    for images, labels in testloader: 
        images = images.cuda()
        labels = labels.cuda()
        ps = net(images)
        i += 1
        top_p, top_class = ps.topk(1, dim=1)
        
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.sum(equals)
        if i% 100 == 0: 
            print(top_class, equals)
            print(sum(equals))
    accuracy = accuracy.item()/(len(testloader)*64)
    print(accuracy)

0
